##Libraries

Parts of code Refereed from https://www.kaggle.com/photon96/nlp-models-reddit/notebook

Pip Install 

In [ ]:
!pip install tensorflow
!pip install transformers
# !pip install keras
# !pip3 install albert-tensorflow
# !pip install torch
# !pip install sentencepiece

     |████████████████████████████████| 1.3MB 9.3MB/s 
     |████████████████████████████████| 890kB 28.4MB/s 
     |████████████████████████████████| 1.1MB 40.8MB/s 
     |████████████████████████████████| 2.9MB 56.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=aa03800a928dcd48d5bb047c1e453cf2215fa18798b086d0db4f3cf7a6044f46
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# Check GPU

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


Assign GPU

In [ ]:
import torch

# Check if GPU is available then telling PyTorch to use the GPU 
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

# Make use of CPU
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


# Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Loading Imports

In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
import torch

Load the pre-trained FinALBERT Model

In [ ]:
from transformers import AlbertTokenizer
from transformers.modeling_albert import AlbertModel, load_tf_weights_in_albert, AlbertPreTrainedModel
from transformers import AlbertForSequenceClassification,AlbertConfig
from transformers.tokenization_bert import BertTokenizer
import torch.nn as nn
from torch.nn import CrossEntropyLoss
VOCAB_FILE = "/content/gdrive/My Drive/ModelFiles/30k-clean.model" # This is the vocab file output from Build Vocab step
tokenizer = AlbertTokenizer.from_pretrained(VOCAB_FILE, do_lower_case=True)
config = AlbertConfig.from_json_file('/content/gdrive/My Drive/ModelFiles/albert_config.json') # Loading ALbert Config File
config.num_labels = 3
model = AlbertForSequenceClassification(config) #Using the transformer ALbert Model
model = load_tf_weights_in_albert(model, config,'/content/gdrive/My Drive/ModelFiles/models_recc2_model.ckpt-best.index')
print(model)
# model.cuda()

Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated


AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(256, 128)
      (token_type_embeddings): Embedding(3, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

#Load the FAANG Dataset

In [ ]:

data = pd.read_csv('/content/gdrive/My Drive/Data/Combined_FAANG_percentage_2.2.csv')
# data = pd.read_csv('/content/gdrive/My Drive/Data/Combined_FAANG_binary_previous.csv')
print("Number of training examples {}".format(len(data)))

Number of training examples 2566826


Limitting Dataset to 1 year

In [ ]:
data.drop(data[data['Date'] < '2019-07-20'].index, inplace = True)
data

,symbol,message,datetime,user,message_id,Date,Time,label
0,AAPL,peak profit last 6 expired option alerts aapl ...,2020-07-19 09:49:35,1442893,229008387,2020-07-19,09:49:35,1
1,AAPL,aapl jul 17 382 50 calls option volume 144 44 ...,2020-07-19 09:47:26,1442893,229008357,2020-07-19,09:47:26,1
2,AAPL,tsla market true bubble territory profitable c...,2020-07-19 09:01:25,1115913,229007569,2020-07-19,09:01:25,1
3,AAPL,aapl analyzed 26 analysts buy consensus 86 ana...,2020-07-19 08:13:00,47688,229006733,2020-07-19,08:13:00,1
4,AAPL,aapl new article dogs dow august 4 adopt ignore,2020-07-19 07:54:05,1555408,229006403,2020-07-19,07:54:05,1
...,...,...,...,...,...,...,...,...
2010966,NFLX,nflx bloated big really consolidation since 10...,2019-07-20 00:13:38,1586407,171184891,2019-07-20,00:13:38,-1
2010967,NFLX,nflx post forever asking proof spent 200k call...,2019-07-20 00:12:03,1557059,171184815,2019-07-20,00:12:03,-1
2010968,NFLX,nflx uhh short take back 270 27 start shipping...,2019-07-20 00:10:42,217259,171184751,2019-07-20,00:10:42,-1
2010969,NFLX,nflx sub 300 wednesday,2019-07-20 00:10:33,1428495,171184744,2019-07-20,00:10:33,-1


Replacing value of Negative label from -1 to 2

In [ ]:
# df_label["label"].replace({-1: 2}, inplace=True) # 0 - neu, 1 - pos, 2 - neg
# data.drop(data[data['label'] == 0].index, inplace = True) 
data["label"].replace({-1: 2}, inplace=True)

In [ ]:
data = data.sample(frac=1)
data

,symbol,message,datetime,user,message_id,Date,Time,label
1945833,NFLX,21 march maddive selloff sector rotation foid ...,2020-03-11 21:22:44,2011874,199716273,2020-03-11,21:22:44,2
216457,AAPL,aapl short position 225 54 20 00,2019-09-12 14:57:06,1601916,177390427,2019-09-12,14:57:06,0
1927007,NFLX,nflx baggies sell pump going back shortly,2020-04-28 16:42:14,1314165,209040750,2020-04-28,16:42:14,2
1693218,FB,fb open 213,2020-05-13 12:16:05,1802585,212267923,2020-05-13,12:16:05,2
1940529,NFLX,nflx leader teen streaming,2020-04-08 17:50:35,1104790,205348240,2020-04-08,17:50:35,0
...,...,...,...,...,...,...,...,...
1972896,NFLX,nflx hope people like throwing away money lol,2019-11-14 14:32:14,1451331,183686619,2019-11-14,14:32:14,1
1926978,NFLX,nflx red rover red rover send cliff,2020-04-28 17:38:13,2727802,209056269,2020-04-28,17:38:13,2
179352,AAPL,best imac amp mac black friday amp cyber monda...,2019-11-28 07:59:06,1765977,185275089,2019-11-28,07:59:06,1
51796,AAPL,aapl money mouth face,2020-05-04 13:29:21,3086074,210222253,2020-05-04,13:29:21,1


In [ ]:
data = data.dropna(subset=['message'])  
data

,symbol,message,datetime,user,message_id,Date,Time,label
1945833,NFLX,21 march maddive selloff sector rotation foid ...,2020-03-11 21:22:44,2011874,199716273,2020-03-11,21:22:44,2
216457,AAPL,aapl short position 225 54 20 00,2019-09-12 14:57:06,1601916,177390427,2019-09-12,14:57:06,0
1927007,NFLX,nflx baggies sell pump going back shortly,2020-04-28 16:42:14,1314165,209040750,2020-04-28,16:42:14,2
1693218,FB,fb open 213,2020-05-13 12:16:05,1802585,212267923,2020-05-13,12:16:05,2
1940529,NFLX,nflx leader teen streaming,2020-04-08 17:50:35,1104790,205348240,2020-04-08,17:50:35,0
...,...,...,...,...,...,...,...,...
1972896,NFLX,nflx hope people like throwing away money lol,2019-11-14 14:32:14,1451331,183686619,2019-11-14,14:32:14,1
1926978,NFLX,nflx red rover red rover send cliff,2020-04-28 17:38:13,2727802,209056269,2020-04-28,17:38:13,2
179352,AAPL,best imac amp mac black friday amp cyber monda...,2019-11-28 07:59:06,1765977,185275089,2019-11-28,07:59:06,1
51796,AAPL,aapl money mouth face,2020-05-04 13:29:21,3086074,210222253,2020-05-04,13:29:21,1


#Splitting Dataset to 90-10

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['message'].values, data['label'].values, test_size=0.1, random_state=42)

Creating train and test dataframes

In [ ]:
df_train = pd.DataFrame(y_train)
print(df_train.value_counts())

df_test = pd.DataFrame(y_test)
print(df_test.value_counts())

1    215016
2    153597
0     94537
dtype: int64
1    23934
2    17026
0    10502
dtype: int64


# Fine Tune

In [ ]:
train = X_train
labels = y_train

Example of Tokenized messages

In [ ]:
# Print the original sentence.
print(' Original: ', train[10])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(train[10]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(train[10])))

 Original:  aapl wtf thousands risk robinhood going cover losses missed profits like robinhood fuck ng
Tokenized:  ['▁a', 'a', 'pl', '▁w', 't', 'f', '▁thousands', '▁risk', '▁robin', 'hood', '▁going', '▁cover', '▁losses', '▁missed', '▁profit', 's', '▁like', '▁robin', 'hood', '▁fuck', '▁', 'ng']
Token IDs:  [15, 286, 14277, 2313, 258, 2513, 2062, 1044, 4854, 8600, 123, 1027, 6661, 1429, 1749, 20, 65, 4854, 8600, 2126, 245, 5563]


In [ ]:
print(data.message.apply(lambda x: len(x)).quantile([0.9]))
MAX_LEN = 160

0.9    156.0
Name: message, dtype: float64


Tokenizing all of the messages and mapping of the tokens to thier word IDs.


In [ ]:
input_ids = []
attention_masks = []

# For each sentence encode_plus would tokenize, add special tokens ([CLS], [SEP]), map tokens to IDs, create attention masks
for text in train:
    
    encoded_dict = tokenizer.encode_plus(
                        text,                      
                        add_special_tokens = True, 
                        max_length = MAX_LEN,           
                        pad_to_max_length = True,
                        return_attention_mask = True,   
                        return_tensors = 'pt',     
                        truncation = True
                   )
    
    # Encoded sentence is being added to a new list
    input_ids.append(encoded_dict['input_ids'])
    
    # Attention masks are added to a new list 
    attention_masks.append(encoded_dict['attention_mask'])

# Lists are converted into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

print('Original: ', train[10])
print('Token IDs:', input_ids[10])

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  aapl wtf thousands risk robinhood going cover losses missed profits like robinhood fuck ng
Token IDs: tensor([    2,    15,   286, 14277,  2313,   258,  2513,  2062,  1044,  4854,
         8600,   123,  1027,  6661,  1429,  1749,    20,    65,  4854,  8600,
         2126,   245,  5563,     3,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 

Training & validation split


In [ ]:
from torch.utils.data import TensorDataset, random_split


# Creating TensorDataset by combining the training inputs
dataset = TensorDataset(input_ids, attention_masks, labels)

# 90-10 train-validation split

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Dividing the training and validation dataset by selecting samples randomly
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{} training samples'.format(train_size))
print('{} validation samples'.format(val_size))

416835 training samples
46315 validation samples


In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    # classification_rep(labels_flat,pred_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
##To print the model training times for each batch
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Set the seed value all over the place to make this reproducible.
import random
def set_random(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

Function to print the classification Report

In [ ]:
from sklearn.metrics import classification_report
def classification_rep(y_test,log_predictions):
  print('-----------',len(log_predictions),log_predictions)
  print('+++++++',len(y_test),y_test)
  log_probs = tf.nn.log_softmax(log_predictions, axis=-1)
  print('=======', log_probs)
  pred_flat = np.argmax(log_probs, axis=1).flatten()
  labels_flat = y_test
  print(classification_report(labels_flat, pred_flat))


Pythong Garbage collection (To free up RAM)

In [ ]:
import gc
gc.collect()

532

Model Training

In [ ]:
def train_model(train_dataloader, optimizer, epochs):
    
    # To store the training and validation loss, validation accuracy, and timings.
    training_stats = []

    # To measure the total training time for the whole run.
    total_t0 = time.time()
    all_preds = []
    all_labels = []
    # For each epoch...
    for epoch_i in range(0, epochs):

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        
        t0 = time.time()

        
        total_train_loss = 0

        #Setting the model to training mode

        model.train()

        
        for step, batch in enumerate(train_dataloader):

            # This block is for progress update of every batch 
            if step % 40 == 0 and not step == 0:
               elapsed = format_time(time.time() - t0)

                # Printing the progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # From the dataloader unpacking the training batch  
            
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            model.zero_grad()        

            #model outputs
            loss, logits = model(b_input_ids, 
                                 attention_mask=b_input_mask, 
                                 labels=b_labels)

            total_train_loss += loss.item()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_train_loss / len(train_dataloader)            
        #Time taken for epoch 
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Setting the model to evaluation mode
        model.eval()

  
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        
        for batch in validation_dataloader:
            # Unpack this validation batch from our dataloader. 

            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                       attention_mask=b_input_mask,
                                       labels=b_labels)

            total_eval_loss += loss.item()
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # The accuracy of test messages is calculated for the batch and then accumulated
            all_preds.extend(logits)
            all_labels.extend(label_ids)
            total_eval_accuracy += flat_accuracy(logits, label_ids)


        # Final accuracy
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)

        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
    classification_rep(all_labels,all_preds)
    return training_stats

def print_training_stats(training_stats):
    pd.set_option('precision', 2)
    df_stats = pd.DataFrame(data=training_stats)
    df_stats = df_stats.set_index('epoch')
    print(df_stats)

Setting hyperparameters and calling the model train function

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import get_linear_schedule_with_warmup, AdamW

from transformers import AlbertForSequenceClassification,AlbertConfig

lr_s = [1e-5]
batch_sizes = [32]
from itertools import product
hyperparameters = list(product(*[lr_s, batch_sizes]))
print(hyperparameters)
training_statistics = []
for lr, batch_size in hyperparameters:
    print(model)
   
    # Assignining Model to GPU
    model.cuda()
    # Creating training and validation DataLoaders
 
    train_dataloader = DataLoader(
                train_dataset, 
                sampler = RandomSampler(train_dataset),
                batch_size = batch_size 
            )


    validation_dataloader = DataLoader(
                val_dataset, 
                sampler = SequentialSampler(val_dataset), 
                batch_size = batch_size
            )
    

    optimizer = AdamW(model.parameters(),
                      lr = lr, 
                      eps = 1e-8, 
                      weight_decay=1e-2 #added this to reduce over-fitting and fix labelling bais
                    )
    

   
    epochs = 2
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)

    seed_val = 42
    set_random(seed_val)
    
    print("Training with hyperparameters: batch size={}, lr={}".format(batch_size, lr))
    training_stats = train_model(train_dataloader, optimizer, epochs)
    training_statistics.append(training_stats)
    print_training_stats(training_stats)

[(1e-05, 32)]
AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(256, 128)
      (token_type_embeddings): Embedding(3, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



              precision    recall  f1-score   support

           0       0.00      0.00      0.00     18860
           1       0.47      0.95      0.63     42812
           2       0.46      0.09      0.15     30958

    accuracy                           0.47     92630
   macro avg       0.31      0.34      0.26     92630
weighted avg       0.37      0.47      0.34     92630

       Training Loss  Valid. Loss  Valid. Accur. Training Time Validation Time
epoch                                                                         
1               1.05         1.04           0.46       1:42:18         0:03:56
2               1.03         1.03           0.47       1:42:17         0:03:56


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Saving Fine-tuned Model

In [ ]:
import os
#defining output directory
output_dir = '/content/gdrive/MyDrive/ModelFiles/1Year_3labels/'

#making new directory
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)
#Model Save
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

Saving model to /content/gdrive/MyDrive/ModelFiles/1Year_3labels/


('/content/gdrive/MyDrive/ModelFiles/1Year_3labels/tokenizer_config.json',
 '/content/gdrive/MyDrive/ModelFiles/1Year_3labels/special_tokens_map.json',
 '/content/gdrive/MyDrive/ModelFiles/1Year_3labels/spiece.model',
 '/content/gdrive/MyDrive/ModelFiles/1Year_3labels/added_tokens.json')

In [ ]:
model_save_name = 'finetuned_Albert1Year3labels.bin'
path = F"//content/gdrive/MyDrive/ModelFiles/1Year_3labels/"+model_save_name
torch.save(model.state_dict(), path)

#Evaluating Performance on Test dataset

In [ ]:
import pandas as pd
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


print('Number of test sentences: {:,}\n'.format(df_test.shape[0]))

# Create test message and label lists
test_sentences = X_test
test_labels = y_test

# Tokenizing all of the messages and mapping the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For each sentence encode_plus would tokenize, add special tokens ([CLS], [SEP]), map tokens to IDs, create attention masks
for sent in test_sentences:

    encoded_dict = tokenizer.encode_plus(
                        sent,                      
                        add_special_tokens = True, 
                        max_length = 160,          
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',     
                   )
    
    
    input_ids.append(encoded_dict['input_ids'])
   
    attention_masks.append(encoded_dict['attention_mask'])

# Converting the lists into tensors
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(test_labels)

  
batch_size = 8  

# Creation of DataLoaders
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

Number of test sentences: 51,462



/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Loading Fine-tuned Model

In [ ]:
from transformers import BertModel, AlbertForSequenceClassification, AdamW, BertConfig
PRE_TRAINED_MODEL_NAME_OR_PATH = '/content/gdrive/MyDrive/ModelFiles/1Year_3labels'
model = AlbertForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME_OR_PATH, num_labels = 3,output_attentions = False, output_hidden_states = False)
model.cuda()

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(256, 128)
      (token_type_embeddings): Embedding(3, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

Prediction on test set


In [ ]:

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

model.eval()


predictions , true_labels = [], []

# Using the prediction dataloader
for batch in prediction_dataloader:
  
  batch = tuple(t.to(device) for t in batch)
  
  b_input_ids, b_input_mask, b_labels = batch
  
  with torch.no_grad():
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Storing the predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 51,462 test sentences...
    DONE.


Printing Classification Report

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
flat_predictions = np.concatenate(predictions, axis=0)

# For every sample selecting the class with highest value
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

flat_true_labels = np.concatenate(true_labels, axis=0)

print("F1 score: {}".format(precision_recall_fscore_support(flat_predictions, flat_true_labels, average='macro')))

F1 score: (0.3550328126887882, 0.30830551743446377, 0.29093763791778987, None)


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(flat_true_labels, flat_predictions,zero_division=1))

              precision    recall  f1-score   support

           0       1.00      0.00      0.00     10502
           1       0.47      0.89      0.62     23934
           2       0.45      0.18      0.25     17026

    accuracy                           0.47     51462
   macro avg       0.64      0.36      0.29     51462
weighted avg       0.57      0.47      0.37     51462



#Company wise Prediction

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/gdrive/My Drive/Combined_FAANG_percentage_2.2.csv')
# data = pd.read_csv('/content/gdrive/My Drive/Data/Combined_FAANG_binary_previous.csv')
df

,symbol,message,datetime,user,message_id,Date,Time,label
0,AAPL,peak profit last 6 expired option alerts aapl ...,2020-07-19 09:49:35,1442893,229008387,2020-07-19,09:49:35,1
1,AAPL,aapl jul 17 382 50 calls option volume 144 44 ...,2020-07-19 09:47:26,1442893,229008357,2020-07-19,09:47:26,1
2,AAPL,tsla market true bubble territory profitable c...,2020-07-19 09:01:25,1115913,229007569,2020-07-19,09:01:25,1
3,AAPL,aapl analyzed 26 analysts buy consensus 86 ana...,2020-07-19 08:13:00,47688,229006733,2020-07-19,08:13:00,1
4,AAPL,aapl new article dogs dow august 4 adopt ignore,2020-07-19 07:54:05,1555408,229006403,2020-07-19,07:54:05,1
...,...,...,...,...,...,...,...,...
2566821,NFLX,get long nflx stop 44 50,2009-09-22 12:21:48,78,591710,2009-09-22,12:21:48,0
2566822,NFLX,nflx awards 1 mill better movie search engine ...,2009-09-21 17:44:20,384,589527,2009-09-21,17:44:20,0
2566823,NFLX,39 mbt 47 swapped nflx,2009-09-21 15:48:19,78,588855,2009-09-21,15:48:19,0
2566824,NFLX,long nflx,2009-09-21 15:39:57,78,588781,2009-09-21,15:39:57,0


In [ ]:
df.drop(df[df['label'] == 0].index, inplace = True) 
df["label"].replace({-1: 0}, inplace=True)

In [ ]:
df = df.drop(df[df['Date'] <= '2020-06-20'].index)
df

,symbol,message,datetime,user,message_id,Date,Time,label
0,AAPL,peak profit last 6 expired option alerts aapl ...,2020-07-19 09:49:35,1442893,229008387,2020-07-19,09:49:35,1
1,AAPL,aapl jul 17 382 50 calls option volume 144 44 ...,2020-07-19 09:47:26,1442893,229008357,2020-07-19,09:47:26,1
2,AAPL,tsla market true bubble territory profitable c...,2020-07-19 09:01:25,1115913,229007569,2020-07-19,09:01:25,1
3,AAPL,aapl analyzed 26 analysts buy consensus 86 ana...,2020-07-19 08:13:00,47688,229006733,2020-07-19,08:13:00,1
4,AAPL,aapl new article dogs dow august 4 adopt ignore,2020-07-19 07:54:05,1555408,229006403,2020-07-19,07:54:05,1
...,...,...,...,...,...,...,...,...
1916645,NFLX,gnus nflx went hitting 40 early stage went way...,2020-06-21 02:18:09,3417009,221638148,2020-06-21,02:18:09,1
1916646,NFLX,nflx analyzed 30 analysts buy consensus 80 ana...,2020-06-21 02:10:00,47688,221637550,2020-06-21,02:10:00,1
1916647,NFLX,spy 4 years still talking obama wow imagine co...,2020-06-21 01:22:23,543250,221633362,2020-06-21,01:22:23,1
1916648,NFLX,spy gld tlt nflx find trading next week video ...,2020-06-21 01:03:29,438774,221631558,2020-06-21,01:03:29,1


In [ ]:
df['symbol'].unique()

array(['AAPL', 'AMZN', 'FB', 'GOOGL', 'NFLX'], dtype=object)

In [ ]:
# Load a trained model and vocabulary that you have fine-tuned
from transformers import AlbertForSequenceClassification,  AdamW, BertConfig
from transformers import  AlbertTokenizer

output_dir = '/content/gdrive/MyDrive/ModelFiles/1YEAR'

model = AlbertForSequenceClassification.from_pretrained(output_dir)
tokenizer = AlbertTokenizer.from_pretrained(output_dir)
model.to(device)

AlbertForSequenceClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(256, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768,

In [ ]:
#To change the company change isin value 
pred_df = df.loc[df['symbol'].isin(['GOOGL'])]
pred_df

,symbol,message,datetime,user,message_id,Date,Time,label
1793223,GOOGL,wmt walmart poised next competitor amazon alwa...,2020-07-14 23:59:39,3454111,227816923,2020-07-14,23:59:39,1
1793224,GOOGL,uber anyone info goog getting food delivery,2020-07-14 23:35:51,3681981,227811719,2020-07-14,23:35:51,1
1793225,GOOGL,goog analyzed 26 analysts buy consensus 96 ana...,2020-07-14 23:18:00,47688,227807482,2020-07-14,23:18:00,1
1793226,GOOGL,aapl amzn fb msft goog las vegas 39 cosmopolit...,2020-07-14 23:07:24,900651,227805071,2020-07-14,23:07:24,1
1793227,GOOGL,uber remember goog attempt smartphones easy th...,2020-07-14 22:45:47,3790554,227799677,2020-07-14,22:45:47,1
...,...,...,...,...,...,...,...,...
1794982,GOOGL,39 see next year later right makes sense buy r...,2020-06-21 01:15:13,2870908,221632669,2020-06-21,01:15:13,1
1794983,GOOGL,spy trump praised justice scalia said wants pu...,2020-06-21 00:57:57,543250,221631017,2020-06-21,00:57:57,1
1794984,GOOGL,sure goog wants roku 39 client base also want ...,2020-06-21 00:44:17,2870908,221629981,2020-06-21,00:44:17,1
1794985,GOOGL,spy come trumper know bad way 39 watching thin...,2020-06-21 00:44:05,543250,221629966,2020-06-21,00:44:05,1


In [ ]:
pred_df.drop(columns = ['datetime', 'user', 'message_id', 'Time'], inplace = True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
def convert_data(sentences,labels):
  
  input_ids = []
  attention_masks = []

  # For each sentence encode_plus would tokenize, add special tokens ([CLS], [SEP]), map tokens to IDs, create attention masks
  for sent in sentences:

      encoded_dict = tokenizer.encode_plus(
                          sent,                     
                          add_special_tokens = True, 
                          max_length = 160,           
                          truncation = True,
                          pad_to_max_length = True,
                          return_attention_mask = True,
                          return_tensors = 'pt',     
                    )
  
      input_ids.append(encoded_dict['input_ids'])
      attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim=0)
  attention_masks = torch.cat(attention_masks, dim=0)
  labels = torch.tensor(labels)
  return input_ids, attention_masks, labels


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset, random_split
import torch.nn.functional as F  #for softmax function 
import numpy as np 
def final_prediction(df):
  input_ids, attention_masks, labels = convert_data(df.message.values,df.label.values)
  batch_size = 8  
  prediction_data = TensorDataset(input_ids, attention_masks, labels)
  prediction_sampler = SequentialSampler(prediction_data)
  prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

  print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

  model.eval()
  predictions , true_labels = [], []
  prediction = np.empty((0,2)) 


  for batch in prediction_dataloader:
    batch = tuple(t for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    with torch.no_grad():
        outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)
        pt_predictions = F.softmax(outputs[0], dim=-1)  
        prediction = np.append(prediction, pt_predictions.detach().cpu().numpy(), axis=0) 

    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    predictions.append(logits)
    true_labels.append(label_ids)

  print('    DONE.')
  pred_labels = np.argmax(prediction, axis=1)
  return predictions,true_labels, pred_labels

In [ ]:
predictions,true_labels, pred_labels = final_prediction(pred_df)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Predicting labels for 1,739 test sentences...
    DONE.


In [ ]:
pred_df['pred_label'] = pred_labels
pred_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,symbol,message,Date,label,pred_label
1793223,GOOGL,wmt walmart poised next competitor amazon alwa...,2020-07-14,1,1
1793224,GOOGL,uber anyone info goog getting food delivery,2020-07-14,1,1
1793225,GOOGL,goog analyzed 26 analysts buy consensus 96 ana...,2020-07-14,1,1
1793226,GOOGL,aapl amzn fb msft goog las vegas 39 cosmopolit...,2020-07-14,1,1
1793227,GOOGL,uber remember goog attempt smartphones easy th...,2020-07-14,1,1
...,...,...,...,...,...
1794982,GOOGL,39 see next year later right makes sense buy r...,2020-06-21,1,0
1794983,GOOGL,spy trump praised justice scalia said wants pu...,2020-06-21,1,1
1794984,GOOGL,sure goog wants roku 39 client base also want ...,2020-06-21,1,0
1794985,GOOGL,spy come trumper know bad way 39 watching thin...,2020-06-21,1,1


In [ ]:
pred_df['label'].value_counts()

1    1279
0     460
Name: label, dtype: int64

In [ ]:
pred_df['pred_label'].value_counts()

1    1511
0     228
Name: pred_label, dtype: int64

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(pred_df['label'].values, pred_df['pred_label'].values,zero_division=1))

              precision    recall  f1-score   support

           0       0.34      0.17      0.22       460
           1       0.75      0.88      0.81      1279

    accuracy                           0.69      1739
   macro avg       0.54      0.52      0.52      1739
weighted avg       0.64      0.69      0.65      1739



In [ ]:
import numpy as np

print(np.mean(pred_df['pred_label'].values))

0.8688901667625072
